In [216]:
# Libraries
import tkinter as tk
from tkinter import messagebox
import serial  
import json
import os
import time
import functools
import numpy as np
from abc import ABC, abstractmethod

In [217]:
# Parametri e variabili globali
usbport = 'COM7' # porta usb
arduino = None # istanza comunicazione di arduino
movements = [] # contiene i movimenti temporanei prima di essere salvati
baud = 38400 #parametro velocità di comunicazione con arduino
start_time = None #usato per calcolare il tempo dallo start del pulsante

## Class functions for discretizing movements

In [218]:
# Classe astratta padre
class Movement(ABC):
    def __init__(self, startTime, endTime):
        self.startTime = startTime
        self.endTime = endTime
        #L=(np.round((self.endTime-self.startTime)/deltaT)+1).astype(np.int64)
        #self.DiscreteMov=None

    @abstractmethod
    def discretize(self):
        pass

# deltaT è un parametro non obbligatorio. Se non viene dato in input di default è 70
# fromPos e toPos sono due list
class LinearMovement(Movement):
    def __init__(self, startTime, endTime, fromPos, toPos, deltaT=65):
        super().__init__(startTime, endTime)
        self.fromPos = fromPos
        self.toPos = toPos
        if deltaT is not None:
            self.deltaT = deltaT
        else:
            self.deltaT=70
        
    def discretize(self):
        
        # Calcola il numero di intervalli discreti
        L = int(np.round((self.endTime - self.startTime) / self.deltaT)) + 1

        # Genera array di tempi discreti per il set corrente di parametri
        tempi_discreti = (np.linspace(self.startTime, self.endTime, L)).astype(np.int64)

        # Interpola le posizioni dei servomotori per il set corrente di parametri
        posizioni_discrete = (np.linspace(self.fromPos[5], self.toPos[5], L)).astype(np.int64)

        
        # Crea una matrice per memorizzare i valori discretizzati per il set corrente di parametri
        matrice_discretizzata = np.vstack((posizioni_discrete, tempi_discreti))

        

        # Interpola le posizioni dei servomotori per il set corrente di parametri
        # Il for è al contrario, va dall'ultimo elemento al primo per mantenere l'ordine del sistema
        for i in range(4,-1,-1):
            nuove_posizioni = (np.linspace(self.fromPos[i], self.toPos[i], L)).astype(np.int64)
            #print(nuove_posizioni)

            # Crea una matrice per memorizzare i valori discretizzati per il set corrente di parametri
            matrice_discretizzata = np.vstack((nuove_posizioni,matrice_discretizzata))

        return matrice_discretizzata.T

## Functions

In [219]:
# This 2 functions maps an input value (in range 0-100) to the corresponding value on the servo, accordingly to the 
# configuration on the json file

def mapping(thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value):
    with open("config.json", "r") as json_file:
    #Load the contents of the JSON file into a Python dictionary
        data = json.load(json_file)
    
    #thumb - big servo
    thumb_big = data["thumb_big"]
    
    #thumb - little servo
    thumb_little = data["thumb_little"]
    
    #index_finger servo
    index_finger = data["index_finger"]

    #middle_finger servo
    middle_finger = data["middle_finger"]
 
    #ring_finger servo
    ring_pinky = data["ring_pinky"]
  
    #forearm servo
    forearm = data["forearm"]

    #All inputs
    input_values = [thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value]
    #All range
    fingers_data = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]
    fingers_data_mapped = []
    i = 0
    for single_finger in fingers_data:
        start = single_finger["range_from"]
        stop = single_finger["range_to"]
        #print(single_finger)
        fingers_data_mapped.append(calculus(input_values[i],start,stop))
        i = i+1
        
    return fingers_data_mapped
        
def calculus(val,start,stop):
    if start==0:
        return int((val/100)*stop)
    else:
        new_stop = stop - start
        return int(((val/100)*new_stop)+start)  

In [220]:
#******************Funzione per salvare un pacchetto o un movimento ***************************
# Struttura di salvataggio: thumb_big, thumb_little, index, middle, ringPinky, forearm, time
# Movement deve essere una list di list (matrice)
def save_movement(name, movement):
    
    # Carica i dati esistenti dal file JSON se esiste
    # Se il file json non esiste, lo crea e ci aggiunge il nuovo movimento
    dati = {}
    if os.path.exists("movements.json"):
        #Se il file esiste, apro il file
        with open("movements.json", 'r') as file_json:
            dati = json.load(file_json)
            
        # Verifica se il nome è già presente nei dati
        if name in dati:
            print(f"Un movimento con il nome '{name}' esiste già. L'aggiunta verrà annullata")
            return False
        else:
            # Aggiungi la nuova matrice ai dati
            dati[name] = movement

            # Salva il dizionario aggiornato in formato JSON nel file specificato
            with open("movements.json", "w") as json_file:
                json.dump(dati, json_file)
                print(f"Movimento '{name}' aggiunto con successo al file")
                return True
    else:
        dati[name] = movement
        with open("movements.json", "w") as json_file:
            json.dump(dati, json_file)
            print(f"Movimento '{name}' aggiunto con successo al file")
            return True

In [221]:
# Apertura connessione seriale
def open_serial_port():
    global arduino 
    try:
        if arduino is None or not arduino.is_open:
            arduino = serial.Serial(port=usbport, baudrate=baud, timeout=.4) 
            time.sleep(2)
            return True
    except serial.SerialException as e:
        print("Error while opening the serial port:", e)
        messagebox.showerror("Error", "Error while opening the serial port")
        return False
        

In [222]:
# Funzione per la validazione dei campi di input della GUI
def on_validate(action, index, value_if_allowed, prior_value, text, validation_type, trigger_type, widget_name):
    if action == '1':  # Inserimento di un carattere
        if text in '0123456789.-+':
            try:
                # Converti il valore in un int
                float_value = int(value_if_allowed)
                # Verifica se il valore è compreso tra 0 e 100
                if 0 <= float_value <= 100:
                    return True
                else:
                    return False
            except ValueError:
                return False
        else:
            return False
    else:
        return True


# Funzione per la validazione del campo di input "time" della GUI (per i valori interi in millisecondi)
def on_validate2(action, index, value_if_allowed, prior_value, text, validation_type, trigger_type, widget_name):
    if action == '1':  # Inserimento di un carattere
        if text in '0123456789.-+':
            try:
                # Converti il valore in un int
                float_value = int(value_if_allowed)
                # Verifica se il valore è compreso tra 0 e 100.000 (100 secondi)
                if 0 <= float_value <= 100000:
                    return True
                else:
                    return False
            except ValueError:
                return False
        else:
            return False
    else:
        return True



In [223]:
# Funzioni usate in FRAME 1

# Funzione per il pulsante esegui movimento nel FRAME1 della GUI
def on_submit(gui_instance):
    packet = []
    for entry in gui_instance.entry_list1:
        try:
            value = int(entry.get())
            packet.append(value)
        except Exception as e:
            messagebox.showerror("Error", "Enter valid numeric values in all boxes")
            print("ERRORE",e)
            return
    
    result = open_serial_port() #apertura seriale
    # Comunicazione ad arduino dei valori
    # Mapping: thumb big - thumb little - index - middle - ring&pinky - forearm
    fingers_data_mapped = mapping(packet[0],packet[1],packet[2],packet[3],packet[4],packet[5])
    # Send values to arduino
    global arduino
    arduino.write(bytearray(fingers_data_mapped))


In [224]:
# Funzioni usate in FRAME 2

# Funzione per il salvataggio dei movimenti in frame2
def on_save(gui_instance,init_list,end_list,time_init,time_end,deltaT):
    
       
    # Controlli (il fatto che siano numeri vengono controllati direttamente sui campi)
    for item in init_list:
        if item.get() == '' or item.get() is None:
            messagebox.showerror("Error", "Fill all init fields")
            return
            
    for item in end_list:
        if item.get() == '' or item.get() is None:
            messagebox.showerror("Error", "Fill all end fields ")
            return
    
    if time_init.get() == '' or time_init.get() is None:
        messagebox.showerror("Error", "Insert an initial time")
        return
    if time_end.get() == '' or time_end.get() is None:
        messagebox.showerror("Error", "Insert a final time")
        return
    
    deltaTInterno = None
    if deltaT.get() == '' or deltaT.get() is None:
        deltaTInterno=65
    else:
        deltaTInterno = deltaT.get()
    
    #Se il controllo è andato bene passo tutto alla classe LinearMovement (trasformando tutti i valori in int prima)
    fromPos = []
    toPos = []
    for item in init_list:
        fromPos.append(int(item.get()))
    
    for item in end_list:
        toPos.append(int(item.get()))
    
    linMov = LinearMovement(int(time_init.get()), int(time_end.get()), fromPos, toPos, int(deltaTInterno))
    matrix_to_save = linMov.discretize()
    
    
    # Funzione on_ok viene chiamata quando si preme ok nella finestra di input per inserire il nome movimento
    def on_ok():
        input_text = entry.get()
        if input_text:
            #salvataggio nel json
            if(save_movement(input_text,matrix_to_save.tolist()) is False):
                messagebox.showinfo("Info", f"Un movimento con questo nome esiste già")
                input_window.destroy()
            else: 
                messagebox.showinfo("Info", f"Hai inserito il movimento: {input_text}")
                input_window.destroy()
            
    # Creazione della finestra di input
    input_window = tk.Toplevel(gui_instance)
    input_window.title("Enter movement name")
    input_window.geometry("400x150")
    
     #Widget Entry per inserire il testo
    entry = tk.Entry(input_window)
    entry.pack(pady=10)

     #Pulsante "OK" per confermare l'input
    ok_button = tk.Button(input_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)


In [225]:
# Funzioni usate in FRAME 3

#legge valori dal movements.json che contiene i movimenti salvati
def read_json():
    try:
        with open("movements.json", 'r') as file:
            data = json.load(file)
        return data
    except FileNotFoundError:
        print("File not found")
        return None
    except json.JSONDecodeError as e:
        print("Parsing error:", e)
        return None

# Esecuzione di un movimento salvato - 
def execute_movement(movement):
    global start_time
    start_time = time.time()
    open_serial_port() #apertura seriale
    
    for packet in range(len(movement)):
        # Comunicazione ad arduino dei valori
        # Mapping: thumb big - thumb little - index - middle - ring&pinky - forearm
        val = movement[packet][6]/1000
        #print("pacchetto")
        #print(val)
        while val > (time.time() - start_time):
            #print(movement[packet][6])
            continue
        
        if packet < len(movement)-1:
            val = movement[packet+1][6]/1000
            if(val < (time.time() - start_time)):
                continue
        
        fingers_data_mapped = mapping(movement[packet][0],movement[packet][1],movement[packet][2],movement[packet][3],movement[packet][4],movement[packet][5])
        
        global arduino
        arduino.write(bytearray(fingers_data_mapped))
            #time.sleep(delay)

def visualize_movement(gui_instance,movement):
    #print(movement)
    
    # Creazione della finestra di input
    input_window = tk.Toplevel(gui_instance)
    input_window.title("Enter movement name")
    input_window.geometry("500x500")
    
    # Creazione del canvas
    canvas = tk.Canvas(input_window, width=500, height=500)
    canvas.pack()

    # Creazione di una scrollbar orizzontale
    scrollbar_x = tk.Scrollbar(input_window, orient="horizontal", command=canvas.xview)
    #scrollbar_x.grid(row=10, column=1, sticky="ew")
    # Configurazione del canvas per l'associazione alla scrollbar orizzontale
    canvas.config(xscrollcommand=scrollbar_x.set)

    # Funzione per l'evento della scrollbar del canvas
    def on_canvas_configure(self):
        canvas.configure(scrollregion=canvas.bbox("all"))
        
    # Configura la scrollbar quando il canvas viene ridimensionato
    canvas.bind("<Configure>", on_canvas_configure)
    
    def fill_canvas(movement, canvas, larghezza_cella, altezza_cella):
        # canvas.delete("all") # pulizia del canvas
        num_righe = len(movement)        # Il numero di righe originali
        num_colonne = len(movement[0])   # Il numero di colonne originali
        r = 0
        c = 0

        # disposizione nel canvas
        for col in range(num_colonne):
            for row in range(num_righe):
                x0 = c * larghezza_cella
                y0 = r * altezza_cella
                x1 = x0 + larghezza_cella
                y1 = y0 + altezza_cella
                canvas.create_text((x0 + x1) // 2, (y0 + y1) // 2, text=str(movement[row][col]))
                r += 1
            c += 1
            r = 0

    fill_canvas(movement,canvas,40,21)
    
    

## GUI

In [ ]:
class GUI(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Hand movement GUI")
        self.geometry("700x700")

        self.columnconfigure(0, weight=1)  # Prima colonna
        self.columnconfigure(1, weight=1)  # Seconda colonna
        
        # Creazione dei frame
        self.frame1 = tk.Frame(self)
        self.frame2 = tk.Frame(self)
        self.frame3 = tk.Frame(self)
        
        # Configurazione dei frame
        self.configure_frame1()
        self.configure_frame2()
        self.configure_frame3()
        
        # Configurazione menù
        self.configure_menu()
        
        
        # Mostra il primo frame all'avvio
        self.show_frame(self.frame1)

    # *************************************** FRAME 1 CONFIGURATION ***************************************************
    def configure_frame1(self):
        
        title = tk.Label(self.frame1, text="Rapid movement",font="8")
        title.grid(row=0,column=1,pady=40,sticky="w")
        validate_cmd = self.frame1.register(on_validate)
        
        
        # Text1
        label1 = tk.Label(self.frame1, text="Thumb - big servo")
        label1.grid(row=1,column=0)
        thumb_big = tk.Entry(self.frame1, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big.insert(0, "Value") 
        thumb_big.configure(justify=tk.CENTER) 
        thumb_big.grid(row=1,column=1)
        
        # Text2
        label2 = tk.Label(self.frame1, text="Thumb - little servo")
        label2.grid(row=2,column=0,padx=10)
        thumb_little = tk.Entry(self.frame1, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little.insert(0, "Value") 
        thumb_little.configure(justify=tk.CENTER) 
        thumb_little.grid(row=2,column=1,pady=5)

        
        # Text3
        label3 = tk.Label(self.frame1, text="Index finger")
        label3.grid(row=3,column=0,padx=10)
        index_finger = tk.Entry(self.frame1, fg='black',validate="key", validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_finger.insert(0, "Value") 
        index_finger.configure(justify=tk.CENTER) 
        index_finger.grid(row=3,column=1,pady=5)
        
        # Text4
        label4 = tk.Label(self.frame1, text="Middle finger")
        label4.grid(row=4,column=0)
        middle_finger = tk.Entry(self.frame1, fg='black',validate="key", 
                                 validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_finger.insert(0, "Value") 
        middle_finger.configure(justify=tk.CENTER) 
        middle_finger.grid(row=4,column=1,pady=5)
        # Text5
        label5 = tk.Label(self.frame1, text="Ring and Pinky")
        label5.grid(row=5,column=0)
        ring_pinky = tk.Entry(self.frame1, fg='black',validate="key", 
                              validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky.insert(0, "Value") 
        ring_pinky.configure(justify=tk.CENTER) 
        ring_pinky.grid(row=5,column=1,pady=5)
        
        # Text6
        label6 = tk.Label(self.frame1, text="Forearm")
        label6.grid(row=6,column=0)
        forearm = tk.Entry(self.frame1, fg='black',validate="key", 
                           validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm.insert(0, "Value") 
        forearm.configure(justify=tk.CENTER) 
        forearm.grid(row=6,column=1,pady=5)
        
        #Valori dei campi innestati nell'istanza self
        self.entry_list1 = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]
        
        # Button
        button1 = tk.Button(self.frame1, text="Execute", command=lambda: on_submit(self))
        button1.grid(row=7,column=1)
        
        

    # **************************************** FRAME 2 CONFIGURATION *************************************************
    def configure_frame2(self):
        
        title = tk.Label(self.frame2, text="Linear movement",font="8")
        title.grid(row=0,column=1,pady=20,sticky="ew")
        validate_cmd = self.frame2.register(on_validate)
        validate_cmd2 = self.frame2.register(on_validate2)
        
        
        # *********** POLLICE - grande ***************************
        # Etichetta 
        label1 = tk.Label(self.frame2, text="Thumb - big servo")
        label1.grid(row=1,column=0,sticky='e')
        
        # entry - init position
        thumb_big_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big_init.insert(0, "Value") 
        thumb_big_init.configure(justify=tk.CENTER) 
        thumb_big_init.grid(row=1,column=1,padx=5)
        
        # entry - end position
        thumb_big_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big_end.insert(0, "Value") 
        thumb_big_end.configure(justify=tk.CENTER) 
        thumb_big_end.grid(row=1,column=2)
        
        
        # *********** POLLICE - piccolo ***************************
        label2 = tk.Label(self.frame2, text="Thumb - little servo")
        label2.grid(row=2,column=0,sticky='e')
        
        # entry - init position
        thumb_little_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little_init.insert(0, "Value") 
        thumb_little_init.configure(justify=tk.CENTER) 
        thumb_little_init.grid(row=2,column=1,padx=5)
        
        # entry - end position
        thumb_little_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little_end.insert(0, "Value") 
        thumb_little_end.configure(justify=tk.CENTER) 
        thumb_little_end.grid(row=2,column=2)

        
        # *********** INDICE ***************************
        label3 = tk.Label(self.frame2, text="Index")
        label3.grid(row=3,column=0,sticky='e')
        
        # entry - init position
        index_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_init.insert(0, "Value") 
        index_init.configure(justify=tk.CENTER) 
        index_init.grid(row=3,column=1,padx=5)
        
        # entry - end position
        index_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_end.insert(0, "Value") 
        index_end.configure(justify=tk.CENTER) 
        index_end.grid(row=3,column=2)
        
        
        # *********** MEDIO ***************************
        label4 = tk.Label(self.frame2, text="Middle")
        label4.grid(row=4,column=0,sticky='e')
        
        # entry - init position
        middle_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_init.insert(0, "Value") 
        middle_init.configure(justify=tk.CENTER) 
        middle_init.grid(row=4,column=1,padx=5)
        
        # entry - end position
        middle_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_end.insert(0, "Value") 
        middle_end.configure(justify=tk.CENTER) 
        middle_end.grid(row=4,column=2)
        
        
        # *********** ANULARE-MIGNOLO ***************************
        label5 = tk.Label(self.frame2, text="Ring-Pinky")
        label5.grid(row=5,column=0,sticky='e')
        
        # entry - init position
        ring_pinky_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky_init.insert(0, "Value") 
        ring_pinky_init.configure(justify=tk.CENTER) 
        ring_pinky_init.grid(row=5,column=1,padx=5)
        
        # entry - end position
        ring_pinky_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky_end.insert(0, "Value") 
        ring_pinky_end.configure(justify=tk.CENTER) 
        ring_pinky_end.grid(row=5,column=2)
        
        
        # *********** AVAMBRACCIO ***************************
        label6 = tk.Label(self.frame2, text="Ring-Pinky")
        label6.grid(row=6,column=0,sticky='e')
        
        # entry - init position
        forearm_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm_init.insert(0, "Value") 
        forearm_init.configure(justify=tk.CENTER) 
        forearm_init.grid(row=6,column=1,padx=5)
        
        # entry - end position
        forearm_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm_end.insert(0, "Value") 
        forearm_end.configure(justify=tk.CENTER) 
        forearm_end.grid(row=6,column=2)
        
        
        # *********** TEMPO ***************************
        label7 = tk.Label(self.frame2, text="Time")
        label7.grid(row=7,column=0,sticky='e')
        
        # entry - init time
        time_init = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd2, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        time_init.insert(0, "Value") 
        time_init.configure(justify=tk.CENTER) 
        time_init.grid(row=7,column=1,padx=5)
        
        # entry - end time
        time_end = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd2, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        time_end.insert(0, "Value") 
        time_end.configure(justify=tk.CENTER) 
        time_end.grid(row=7,column=2)
        
        # *********** DELTA T ***************************
        label8 = tk.Label(self.frame2, text="DeltaT")
        label8.grid(row=8,column=0,sticky='e')
        
        # entry - init time
        deltaT = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd2, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        deltaT.insert(0, "Value") 
        deltaT.configure(justify=tk.CENTER) 
        deltaT.grid(row=8,column=1,padx=5)
        
        #array dei valori di partenza
        init_list = [thumb_big_init,thumb_little_init,index_init,middle_init,
                     ring_pinky_init,forearm_init]
        
        #array dei valori fine
        end_list = [thumb_big_end,thumb_little_end,index_end,middle_end,
                    ring_pinky_end,forearm_end]
        
        # Buttons
        button1 = tk.Button(self.frame2, text="Save", command=lambda: on_save(self,init_list,end_list,time_init,time_end,deltaT))
        button1.grid(row=9,column=1,sticky='e',pady=15)
        

    # *************************************** FRAME 3 CONFIGURATION **********************************************
    def configure_frame3(self):
        #print("Sono in frame 3")
        num_columns = 5  # Definisci il numero fisso di colonne
        for i in range(num_columns):
            self.frame3.columnconfigure(i, weight=1)
        title = tk.Label(self.frame3, text="Saved movements", font="8")
        title.grid(row=0, column=5, pady=20)  # Modifica la posizione e allarga il titolo su due colonne
        
        movements_dict = read_json()
        if movements_dict is None: 
            print("Errore, json vuoto")
            return

        index = 1
        for key, value in movements_dict.items():
            label = tk.Label(self.frame3, text=str(key))
            label.grid(row=index, column=5, padx=10, pady=5, sticky="w")  # Aggiungi il flag sticky per allineare a sinistra
            textt = str(key)

            button = tk.Button(self.frame3, text="Execute", command=functools.partial(execute_movement, value))
            button.grid(row=index, column=5, padx=10, pady=5, sticky="e")  # Aggiungi il flag sticky per allineare a destra
            button2 = tk.Button(self.frame3, text="Visualize", command=functools.partial(visualize_movement,self,value))
            button2.grid(row=index, column=6, padx=10, pady=5, sticky="e") 
            index += 1

            
    
        
    # Funzione per configurare il menù della GUI in alto a sinistra
    def configure_menu(self):
        # Creazione del menu
        menubar = tk.Menu(self)
        self.config(menu=menubar)
        frame_menu = tk.Menu(menubar, tearoff=0)
        frame_menu.add_command(label="Rapid movement", command=lambda: self.show_frame(self.frame1))
        frame_menu.add_command(label="Linear movement", command=lambda: self.show_frame(self.frame2))
        frame_menu.add_command(label="Saved movements", command=lambda: self.show_frame(self.frame3))
        frame_menu.add_command(label="Shutdown", command=lambda: self.close())

        menubar.add_cascade(label="Menù", menu=frame_menu)

    # Nasconde tutti i frame e mostra solo quello specificato
    def show_frame(self, frame):
        #Riaggiorno il frame3
        self.configure_frame3()
        
        self.frame1.grid_forget()
        self.frame2.grid_forget()
        self.frame3.grid_forget()
        frame.grid()
       
    #Chiusura programma e com. seriale
    def close(self):
        #print("Sto chiudendo tutto")
        global arduino
        if arduino is not None:
            arduino.close()
            print("Closed serial connection")
        self.destroy()         
        
        
if __name__ == "__main__":
    app = GUI()
    app.mainloop()


In [56]:
arduino.close()

AttributeError: 'NoneType' object has no attribute 'close'

In [32]:
movements.clear()